In [1]:
import pandas as pd
import requests
from datetime import datetime
from dateutil import parser

from urllib import parse
from data_pull import data_pull_date
from data_pull import data_pull_all

### File Codes

In [2]:
# Energy Generation Type (All)
# "f93d1835-75bc-43e5-84ad-12472b180a98"
# https://data.nationalgrideso.com/carbon-intensity1/historic-generation-mix/r/historic_gb_generation_mix

# Energy Demand (2019)
# dd9de980-d724-415a-b344-d8ae11321432
# https://data.nationalgrideso.com/demand/historic-demand-data/r/historic_demand_data_2019

# Energy Cost (2019-2020 Financial Year)
# 6a03c415-475f-4005-bf55-b9108291c511
# https://data.nationalgrideso.com/trade-data/historic-gtma-grid-trade-master-agreement-trades-data/r/historic_gtma_trades_data_-_apr_2019_-_march_2020

# System Frequency (Dec 2019)
# f0933bdd-1b0e-4dd3-aa7f-5498df1ba5b9    
# https://data.nationalgrideso.com/system/system-frequency-data/r/december_2019_-_historic_frequency_data

# Metered Wind Power (2019 - 2020)
#c36868d9-9d43-47ce-ac3e-de69f0bb6222
#https://data.nationalgrideso.com/generation/monthly-operational-metered-wind-output/r/monthly_operational_metered_wind_output_2019-2020

### Load Generation Data

In [9]:
date_start = "2019-01-01T00:00:00.000Z"
date_end = "2019-12-31T23:59:59.999Z"
entry_limit = '100000'

datacode1 = "f93d1835-75bc-43e5-84ad-12472b180a98"

## Load Data
gen = data_pull_date(datacode1,date_start,date_end,entry_limit)
gen['DATETIME'] = pd.to_datetime(gen['DATETIME'], format="%Y/%m/%d %H:%M:%S")

## Check Data Start and End times
start_time = gen.loc[0,"DATETIME"]
start_str = "Start Time: {}".format(start_time)
print(start_str)

idx = gen.index[-1]
end_time = gen.loc[idx,"DATETIME"]
end_str = "End Time: {}".format(end_time)
print(end_str)

# period = datetime.strptime("00:30:00", '%H:%M:%S').time()
# period_str = "Period: {}".format(period)
# print(period_str)     
# print(gen.size)


## Save Data
filename = gen.loc[0,"DATETIME"]
filename = "Data\\Generation\\gen_" + filename.strftime("%Y-%m") + ".csv"
gen.to_csv(filename, sep='\t')
#print(gen.dtypes)

## Seperate Wind Data by Month and save each
T = 48 #Number of 30 min in day
idx_list = gen.index[0::T] # Find index for each day
dtm = gen.loc[idx_list,"DATETIME"] # Get each date of measurement

dtm_unique = []
for ii in dtm:
    dtm_unique.append(ii.strftime("%Y-%m"))
dtm_unique = set(dtm_unique) # Find each unique month
# print(dtm_unique)


## Save Data
dtm = gen["DATETIME"].dt.strftime("%Y-%m") # Get all dates in wind dataset
for ii in dtm_unique:    
    date_match = [ii==j for j in dtm]
    gen_month =  gen.loc[date_match,:] #Extract and save all data for each unique month

    filename = "Data\\Generation\\gen_" + ii + ".csv"
    gen_month.to_csv(filename, sep='\t')

Start Time: 2019-01-01 00:00:00
End Time: 2019-12-31 23:30:00


###  Load Demand Data

In [4]:
datacode2 = "dd9de980-d724-415a-b344-d8ae11321432"
entry_limit = '100000'

## Load Data
dem = data_pull_all(datacode2,entry_limit)
dem['SETTLEMENT_DATE'] = pd.to_datetime(dem['SETTLEMENT_DATE'], format="%Y/%m/%d %H:%M:%S")

## Check Data Start and End times
start_time = dem.loc[0,"SETTLEMENT_DATE"]
start_str = "Start Time: {}".format(start_time)
print(start_str)

idx = dem.index[-1]
end_time = dem.loc[idx,"SETTLEMENT_DATE"]
end_str = "End Time: {}".format(end_time)
print(end_str)

# period = datetime.strptime("00:30:00", '%H:%M:%S').time()
# period_str = "Period: {}".format(period)
# print(period_str)   
# print(dem.size)



## Seperate Demand Data by Month and save each
T = 48 #Number of 30 min in day
idx_list = dem.index[0::T] # Find index for each day
dtm = dem.loc[idx_list,"SETTLEMENT_DATE"] # Get each date of measurement

dtm_unique = []
for ii in dtm:
    dtm_unique.append(ii.strftime("%Y-%m"))
dtm_unique = set(dtm_unique) # Find each unique month
# print(dtm_unique)


## Save Data
dtm = dem["SETTLEMENT_DATE"].dt.strftime("%Y-%m") # Get all dates in wind dataset
for ii in dtm_unique:    
    date_match = [ii==j for j in dtm]
    dem_month =  dem.loc[date_match,:] #Extract and save all data for each unique month

    filename = "Data\\Demand\\dem_" + ii + ".csv"
    dem_month.to_csv(filename, sep='\t')

Start Time: 2019-01-01 00:00:00
End Time: 2019-12-31 00:00:00


### Load Cost Data

In [5]:
## Really complicated to handle

# datacode3 = "6a03c415-475f-4005-bf55-b9108291c511"
# cst = data_pull_all(datacode3,entry_limit)

# start_time = parser.parse(cst.loc[0,"Start Time"])
# start_str = "Start Time: {}".format(start_time)
# print(start_str)

# idx = cst.index[-1]
# end_time = parser.parse(cst.loc[idx,"Start Time"])
# end_str = "End Time: {}".format(end_time)
# print(end_str) 

# cst.dtypes

### Load Frequency Data

In [6]:
# datacode4 = "f0933bdd-1b0e-4dd3-aa7f-5498df1ba5b9"
# entry_limit = '3000000'

# freq = data_pull_all(datacode4,entry_limit)
# freq['dtm'] = pd.to_datetime(freq['dtm'], format="%Y/%m/%d %H:%M:%S")

# start_time = freq.loc[0,"dtm"]
# start_str = "Start Time: {}".format(start_time)
# print(start_str)

# idx = freq.index[-1]
# end_time = freq.loc[idx,"dtm"]
# end_str = "End Time: {}".format(end_time)
# print(end_str)

# period = freq.loc[1,"dtm"] - start_time
# period_str = "Period: {}".format(period)
# print(period_str)  
# print(freq.size)
# #freq.dtypes

### Load Wind Data

In [7]:
entry_limit = '100000'
datacode5 = "c36868d9-9d43-47ce-ac3e-de69f0bb6222"

## Load Wind Data
wind = data_pull_all(datacode5,entry_limit)
wind['Sett_Date'] = pd.to_datetime(wind['Sett_Date'], format="%Y/%m/%d %H:%M:%S")


## Check Data Start and End times
start_time = wind.loc[0,"Sett_Date"]
start_str = "Start Time: {}".format(start_time)
print(start_str)

idx = wind.index[-1]
end_time = wind.loc[idx,"Sett_Date"]
end_str = "End Time: {}".format(end_time)
print(end_str)


## Seperate Wind Data by Month and save each
T = 48 #Number of 30 min in day
idx_list = wind.index[0::T] # Find index for each day
dtm = wind.loc[idx_list,"Sett_Date"] # Get each date of measurement

dtm_unique = []
for ii in dtm:
    dtm_unique.append(ii.strftime("%Y-%m"))
dtm_unique = set(dtm_unique) # Find each unique month
# print(dtm_unique)


## Save Data
dtm = wind["Sett_Date"].dt.strftime("%Y-%m") # Get all dates in wind dataset
for ii in dtm_unique:    
    date_match = [ii==j for j in dtm]
    wind_month =  wind.loc[date_match,:] #Extract and save all data for each unique month

    filename = "Data\\Wind\\wind_" + ii + ".csv"
    wind_month.to_csv(filename, sep='\t')

Start Time: 2019-04-01 00:00:00
End Time: 2020-03-31 00:00:00
